In [2]:
import numpy as np
import pandas as pd
from surprisememore import UndirectedGraph
import os

In [ ]:
os.chdir("C:\\")

# Semilla global para reproducibilidad
SEED = 123  

# Define solo el nombre del archivo
input_filename = 'archivo.csv'
input_filename_f = 'archivo_flujo.csv'

# Construye la ruta completa con la carpeta específica
full_path = os.path.join('C:\\', input_filename)
full_path_f = os.path.join('C:\\', input_filename_f)

edges = pd.read_csv(full_path,sep=';')
edges.iloc[:, :2] -= 1

# Crear la matriz de adyacencia basada en los nodos y pesos en edges_df
num_nodos = max(edges['nod_sal'].max(), edges['nod_ent'].max()) + 1
adjacency_matrix = np.zeros((num_nodos, num_nodos))

# Rellenar la matriz de adyacencia con los valores de 'weight' del DataFrame
for _, row in edges.iterrows():
    adjacency_matrix[row['nod_sal'], row['nod_ent']] = row['weight']

if (edges['weight'] == 1).all():
    adjacency_matrix_mod = adjacency_matrix
else:
    adjacency_matrix_mod = np.zeros((num_nodos, num_nodos))
    for _, row in edges.iterrows():
        adjacency_matrix_mod[row['nod_sal'], row['nod_ent']] = 1

flujo = pd.read_csv(full_path_f,sep=';', header=None)
flujo = np.array(flujo)

nodos = pd.Index(range(max(edges['nod_ent'].max(), edges['nod_sal'].max()) + 1))
K_in = pd.Series(edges.groupby('nod_ent')['weight'].count()).reindex(nodos, fill_value=0)
K_out = pd.Series(edges.groupby('nod_sal')['weight'].count()).reindex(nodos, fill_value=0)

m = len(edges)
n = len(adjacency_matrix)


alpha_values =  np.linspace(1, 0, 101)

# Lista para almacenar los resultados
resultados = []

for alpha in alpha_values:
    A_F = alpha * adjacency_matrix + (1 - alpha) * flujo
    A_Fsim = A_F + A_F.T
    adj_mat = np.round(A_Fsim).astype(int)
    graph = UndirectedGraph(adj_mat)
    if np.max(adj_mat) == 1:
        graph.run_discrete_community_detection(weighted=False,
                                           num_sim=2)
    else:
        graph.run_discrete_community_detection(weighted=True,
                                           num_sim=2)
        
    Q = 0.0
    for i in range(n):
        for j in range(n):
            if graph.solution[i] == graph.solution[j]: # Misma comunidad
                Q += pd.DataFrame(adjacency_matrix_mod).iloc[i, j] - (K_out[i] * K_in[j] / m)      
    Q /= m

    # Convertir la solución a un formato de lista (debería ser una lista de números)
    solucion = graph.solution
    
    # Asegurarnos de que los valores de la solución se agreguen como columnas separadas
    solucion_dict = {f"x_{i+1}": solucion[i] for i in range(len(solucion))}
    
    # Añadir los valores de alpha y modularidad a la fila
    fila_resultado = {"alpha": alpha, "modularidad": Q, **solucion_dict}
    
    # Añadir la fila a la lista de resultados
    resultados.append(fila_resultado)

# Convertir la lista de resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Buscar el valor máximo de la columna 'modularidad'
max_modularidad = df_resultados['modularidad'].max()

# Obtener el primer valor de la columna 'modularidad'
primer_valor = df_resultados['modularidad'].iloc[0]

# Comparar el máximo con el primer valor y agregar una columna de comparación
comparacion = 1 if max_modularidad > primer_valor else 0

# Crear la fila con el valor máximo y la comparación
fila_maximo = {'alpha': 'Max', 'modularidad': max_modularidad, 'comparacion': int(comparacion)}

# Añadir la fila con el valor máximo al DataFrame
df_resultados = pd.concat([pd.DataFrame([fila_maximo]), df_resultados], ignore_index=True)

# Reemplazar inf y NaN por un valor válido, por ejemplo, 0
df_resultados['comparacion'] = df_resultados['comparacion'].replace([np.inf, -np.inf, np.nan], 0)

# Convertir la columna 'comparacion' a enteros
df_resultados['comparacion'] = df_resultados['comparacion'].astype(int)

##################################
# Definir la ruta de la carpeta de salida
output_folder = r"C:\Redes\Comunidades_"

file_base = os.path.splitext(os.path.basename(input_filename))[0]
# Construir el nombre completo del archivo con file_base
output_file = os.path.join(output_folder, f"{file_base}_surprise.csv")

# Guardar el DataFrame en el archivo CSV
df_resultados.to_csv(output_file, index=False, sep=';', quoting=1)
##################################

